<a href="https://colab.research.google.com/github/albachiarabellaroba/HW3---Group-16/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Homework 3 - Places of the world**

In [1]:
from bs4 import BeautifulSoup
import bs4
import requests
import os.path
import lxml
import re
import codecs
import csv
from datetime import datetime
from tqdm import tqdm
import time


In [2]:
from google.colab import drive
drive.mount('/content/drive/') 

Mounted at /content/drive/


## 1. Data collection

### 1.1. Get the list of places

In [ ]:
linklist = open('/content/drive/MyDrive/ADM_HW3/linklist.txt', 'w')

for i in tqdm(range(1, 401)):
    link = 'https://www.atlasobscura.com/places?page='+str(i)+'&sort=likes_count'
    cont = requests.get(link)
    soup = BeautifulSoup(cont.content, features='lxml')
    for a in soup.find_all('a', class_='col-md-3 nav-card'):
        linklist.write(a.get('href')+'\n')
        
linklist.close()

100%|██████████| 400/400 [02:09<00:00,  3.10it/s]


### 1.2. Crawl places

In [3]:
for i in tqdm(range(1, 401)):
    cart = '/content/drive/MyDrive/ADM_HW3/Html_page/page_{}'.format(i)
    os.makedirs(cart)
    
    f = open('/content/drive/MyDrive/ADM_HW3/linklist.txt', 'r', encoding='utf8')
    l = f.readlines()[(i-1)*18:(i)*18]
    l = [i.rstrip() for i in l]
    f.close()

    m = 1+18*(i-1)

    for link in l:
      link = 'https://www.atlasobscura.com' + link
      html = requests.get(link)

      name = '{}/{}.html'.format(cart, i)
      text = open(name, 'w', encoding='utf8')
      text.write(html.text)
      text.close()

      m=m+1


100%|██████████| 400/400 [12:24<00:00,  1.86s/it]


### 1.3 Parse downloaded pages

## 2. Search Engine

### 2.1.1 Create your index!

### 2.1.2 Execute the query

### 2.2 Conjunctive query & Ranking score

### 2.2.1 Inverted index

### 2.2.2 Execute the query

## 3. Define a new score!

## 4. Visualizing the most relevant places

## 5. BONUS: More complex search engine

## 6. Command line question

## 7. Theoretical question